DISC SCORE METRIC

In [ ]:
import tensorflow.keras.backend as K

def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


Nested U-Net (U-Net++) Training

In [ ]:
def nested_unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)

    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    up2 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv3)
    concat2 = layers.concatenate([conv2, up2])
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(concat2)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up1 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv4)
    concat1 = layers.concatenate([conv1, up1])
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(concat1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])

    return model
nested_unet = nested_unet_model()
nested_history = nested_unet.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test, y_test))

nested_unet.save_weights('nested_unet_weights.h5')

nested_eval = nested_unet.evaluate(X_test, y_test)
print("Nested U-Net DICE Score:", nested_eval[1])


Attention U-Net Training

In [ ]:
def attention_unet_model(input_size=(256, 256, 3)):
    # Implement the Attention U-Net architecture here
    inputs = layers.Input(input_size)

    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Add attention mechanism here
    up2 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv3)
    concat2 = layers.concatenate([conv2, up2])
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(concat2)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up1 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv4)
    concat1 = layers.concatenate([conv1, up1])
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(concat1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])

    return model

# Train the Attention U-Net model
attention_unet = attention_unet_model()
attention_history = attention_unet.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test, y_test))

# Save the model weights
attention_unet.save_weights('attention_unet_weights.h5')

# Evaluate the model
attention_eval = attention_unet.evaluate(X_test, y_test)
print("Attention U-Net DICE Score:", attention_eval[1])


MODEL COMPARISON

In [ ]:
print("Nested U-Net DICE Score:", nested_eval[1])
print("Attention U-Net DICE Score:", attention_eval[1])

if nested_eval[1] > attention_eval[1]:
    print("Nested U-Net performs better.")
else:
    print("Attention U-Net performs better.")
